# Breaking SIDH

We are going to implement a special case of the SIDH attack.

You will need a library for computing $(2,2)$-isogenies. If you haven't installed it yet, run the cell below to download the [ThetaIsogenies](https://github.com/ThetaIsogenies/two-isogenies) library in the current folder.

In [ ]:
import zipfile, sys, urllib.request
ar, res = urllib.request.urlretrieve('https://github.com/ThetaIsogenies/two-isogenies/archive/refs/heads/main.zip')
with zipfile.ZipFile(ar, 'r') as zip_ref:
    zip_ref.extractall()

Now you can load the library in this notebook by evaluating this

In [1]:
sys.path.append('./two-isogenies-main/Theta-SageMath')
from theta_structures.couple_point import CouplePoint
from theta_isogenies.product_isogeny_sqrt import EllipticProductIsogenySqrt
from theta_isogenies.product_isogeny import EllipticProductIsogeny

## Setting up parameters to make the attack simpler

To run the attack in its purest form, without extra exhaustive searches, we need an integer $p$ with the following properties:

- $p = 2^a · 3^b - 1$ is prime, and
- $d = 2^a - 3^b = x^2 + y^2 > 0$ for some $x,y$

Find $a,b,d,x,y$ with thes properties. Taking $b$ in the ≈30's should be sufficient for a nice demonstration.

Now take the elliptic curve $E_0 : y^2 = x^3 + x \;/ \mathbb{F}_{p^2}$

Construct torsion bases $〈P_2,Q_2〉 = E_0[2^a]$ and $〈P_3,Q_3〉 = E_0[3^b]$

## Setting up the SIDH challenge

Construct a random **secret key** point $K$ of order $3^b$

Now compute the isogeny $φ : E_0 \to E_0/〈K〉$, let $E_1 := E_0/〈K〉$

Compute the **SIDH public key** $(R_2, S_2) = (φ(P_2), φ(Q_2))$

## Breaking SIDH

Our goal is to recover the secret key $K$ (or any other point generating the same cyclic subgroup $〈K〉$) from the public interpolation data $(P_2, Q_2, φ(P_2), φ(Q_2))$. To this end, we construct a Kani square on the secret isogeny $φ$.

If not done already, find $x,y$ such that $x^2 + y^2 = 2^a - 3^b = d$.

Construct an endomorphism $α ∈ \mathrm{End}(E_0)$ of degree $d$

Let $α':E_1 \to E_2$ be the pushforward of $α$ under $φ$ and $φ':E_0\to E_2$ the pushforward of $φ$ under $α$ (note: you don't have enough data to actually compute them yet).

Construct the isogeny $Φ : E_0 × E_1 \to E_0 × E_2$ given by the isogeny matrix $\pmatrix{\hatα & \hat φ\\ -φ' & α'}$

Warning: the codomain of $Φ$ computed by the library will only be isomorphic to $E_0 × E_2$.

Now evaluate $Φ$ on a well chosen point of $E_0 × E_1$ to get a generator of the kernel of $φ$...

Warning: because the codomain of $Φ$ is defined up to isomorphisms, you may need to try the non-trivial automorphisms of $E_0$ to find the right kernel